In [1]:
#import packages etc
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy
import re
from elasticsearch import Elasticsearch
matplotlib.rc('font', **{'size': 15})
from elasticsearch.helpers import scan
import time
import operator

In [2]:
#declare variables
print("Declaring variables.")

#epoch times of start and end of time interval. 
#first week of february. monday 30. Jan to sunday 5. feb.
#found using epochconverter.com
#epoch time must be in milliseconds
epoch_start = 1485730800000 #jan 30th 00.00.00
epoch_end = 1486335599000 #feb 5th 23.59.59
#epoch_end = 1485820799000 # jan30th 23.59.59 (one day for testing)

loop_time = epoch_start #time variable in loop

#loop time increments . one minute? five minutes?
increment = 5 * 60 * 1000; #milliseconds

#thresholds for defining an error
failure_rate = .9 #failure rate threshold. Q: what is the threshold of red cells in DDM dashboard?
file_thresh = 10 #number of files threshold.
persist_thresh = 15 * 60 * 1000; #milliseconds. time needed before alert is sent.

#number of alarms
alarms = 0


#some kind of dictionary with pairs of sites, and some variable keeping track of time
#if pairs stay on this list too long an alarm is raised {[source,destination]:loop_time}
observed_endpoints = {}

#dict of when an alert was made and what pair is failing {when: [source,receiver]}
print("Done.")

Declaring variables.
Done.


In [3]:
#Elasticsearch querries
print("Defining ElasticSearch querries.")
query_done = {
    "size": 0,
    "query": {
    "bool": {
    "must": [
    {
        "range": {
        "@timestamp": {
            "gte": epoch_start,
            "lte": epoch_end,
            "format": "epoch_millis"
            }
        }
    },
        {"term": {"event_type": "transfer-done"}},
        {"term": {"payload.scope": "data16_13TeV"}}
    ]
    }
    }
}
query_failed = {
    "size": 0,
    "query": {
    "bool": {
    "must": [
    {
        "range": {
            "@timestamp": {
            "gte": epoch_start,
            "lte": epoch_end,
            "format": "epoch_millis"
             }
            }
           },
          {"term": {"event_type": "transfer-failed"}},
          {"term": {"payload.scope": "data16_13TeV"}}
        ]
        }
    }
}


#make scrolls
print("Defining scrolls.")
# make scrolls
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60) 
my_index = "rucio-events-2017*"
scroll_done = scan(es, query=query_done, index=my_index, scroll='5m', timeout="5m", size=100)
scroll_errors = scan(es, query=query_failed, index=my_index, scroll='5m', timeout="5m", size=100)


print("Done.")

Defining ElasticSearch querries.
Defining scrolls.
Done.
